# Deep Deterministic Policy Gradients Agent for Step Response

Trains a [Deep Deterministic Policy Gradients](https://stable-baselines.readthedocs.io/en/master/modules/ddpg.html) model to solve a simple step-response problem in the shared Bullet simulation. A [Tensorboard](https://www.tensorflow.org/tensorboard) visualisation is hosted [**here**](http://0.0.0.0:6006/#scalars) in order to track training progress.

## Setup Environment and start Training

In [ ]:
# Set matplotlib to inline mode and import necessary components
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

import gym
import os
import numpy as np

# These imports witll throw a lot of warnings, since we are using Python 3.7 instead of 3.8
from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines import DDPG

import gym_jointcontrol
import rospy

import time

# Instantiate gym environment and reset it
env = gym.make('jointcontrol-v0', jointidx=0)
env.reset()

# Start tensorboard to allow for visualisation of training process
os.system("tensorboard --logdir /training_tensorboard/ --host 0.0.0.0 --port 6006 &")

# Create DDPG noise objects. They are scaled to [-1, 1] in the shape [numParams,]
n_actions = env.action_space.shape[-1]
param_noise = None
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

# Instantiate model and start training
model = DDPG(MlpPolicy, env, tensorboard_log="/training_tensorboard/", verbose=1, param_noise=param_noise, action_noise=action_noise)
model.learn(total_timesteps=10000)

## Test trained Model

The trained model can then be tested using the controllerTest module from the jointcontrol package. This module allows for step response and stability testing of the resulting controller and generates a test report.

In [ ]:
# Import controllerTest module from jointcontrol package
import sys
sys.path.append("/catkin_ws/src/jointcontrol/scripts")
from controllerTest import *

# Reset environment, instantiate controllerTest and visualise results
env.reset()
test = controllerTest(env, model, "Deep Deterministic Policy Gradients")
test.plotResults(gui=True)